In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

# Begin by creating a list of h values corresponding to temperatures

temps_h = np.linspace(100, 700, num = 600) # Set up temperature range [K]

np.savetxt('output2.csv', temps_h, delimiter=',', fmt='%d')

# Variables for zeta
M = 0.1998 # Mach number - nozzle inlet
T0 = 2066.9749 # Adiabatic flame temp - injector prop temp [K]
gamma = 1.2737 # nozzle inlet

zeta = []

for i in range(len(temps_h)):
    zeta.append(1 / ( ( 0.5 * temps_h[i] / T0 * ( 1 + (gamma - 1) / 2 * M**2 ) + 0.5 )**0.68 * ( 1 + (gamma - 1) / 2 * M**2)**0.12))

# Variables for h
Dt = 22.02 / 1000
mu = 6.794e-05
Cp_flu = 1.924*1000
Pr = 0.5525
P1 = 2000000
c_star =  1391.15
r_c = 21.82 / 1000
At = math.pi * (Dt / 2)**2
Ac = math.pi * (38.13 / 2000)**2

h = []

for i in range(len(zeta)):
    h.append( 0.026 / Dt**0.2 * (mu**0.2 * Cp_flu / Pr**0.6) * (P1 / c_star )**0.8 * (Dt/r_c)**0.1 * (At/Ac)**0.9 * zeta[i])

# How to find corresponding temperature

np.savetxt('output.csv', h, delimiter=',', fmt='%d')

y = 650
h_test = min(h, key=lambda x: abs(x - y))
print(h_test)

plt.figure()
plt.plot(temps_h, h)
plt.xlabel("Temperature (K)")
plt.ylabel("Convective Heat Transfer Coefficient (W/m^2K)")
plt.grid(visible=True, which='both', axis='both', color='lightsteelblue', linestyle='--', linewidth=0.5)

# Heat Transfer - variable h

# Cylinder combustion chamber
L = 75.77 / 1000
ID = 38.13 / 1000
OD = 100 / 1000

# Material properties matching Ansys - all of these change with temperature
rho = 8920
k = 391.1
Cp = 385

# Propellant and initial temperature
Ti = 298
Tfl = 2067

A = math.pi * ID * L # Internal chamber surface area [m2]
A2 = math.pi * OD * L # Outer chamber surface area [m2]

# Split the combustion chamber into equal segments
r = 20
sl_size = (OD - ID) / (2 * r)

# Calculate slice radii
sliceR = []
sliceD = []
V = []
m = []
for i in range(r):
    sliceR.append(ID / 2 + sl_size * (i + 1))
    d = ((ID / 2 + sl_size * (i + 1)) * 2)
    sliceD.append(d)
    if i == 0:
        v = math.pi * (d/2)**2 * L - math.pi * (ID/2)**2 * L
    else:
            v = math.pi * (d/2)**2 * L - math.pi * (sliceD[i - 1] / 2)**2 * L
    V.append(v) # Volume [m3]
    m.append(v * rho)

h_bartz = 4116.3 # Bartz
h_nat = 10

t = np.linspace(0, 5, num = 15000)

Ti = np.linspace(290, 290, num = r)

temps2 = pd.DataFrame({"t = 0": Ti})



for i in range(len(t)):
    if i == 0:
        continue
        
    else:
        # Start an empty list to record the temperature of each position for a specific moment in time
        T = []
        
        # Convection into first slice
        Q_conv = (min(h, key=lambda x: abs(x - temps2.iloc[0][i-1]))) * A * (Tfl - temps2.iloc[0][i-1])
        dT_conv = Q_conv * (t[i] - t[i - 1]) / (m[0] * Cp)
        T_no_cond = temps2.iloc[0][i-1] + dT_conv # Inner wall temperature if no heat is removed by conduction
        
        # Conduction from first slice into second
        dT_cond = T_no_cond - temps2.iloc[1][i-1] # Temp difference between each slice
        Q_cond = (2 * math.pi * k * L * dT_cond) / (math.log(sliceD[1] / sliceD[0]))
        dT_slice1 = (Q_conv - Q_cond) * (t[i] - t[i - 1]) / (m[0] * Cp)
        T_slice1 = temps2.iloc[0][i-1] + dT_slice1
        T.append(T_slice1)
        
        Q_cond_vec = []
        
        # Use another for loop to iterate through the centre slices where heat transfer modes are conduction only
        for j in range(r - 2):
            if j == 0:
                # Temperature change in slice x from conduction in only - no conduction out
                dT_slice_int = Q_cond * (t[i] - t[i - 1]) / (m[j + 1] * Cp) # Mass value will change depending on slice
                T_cond_int = temps2.iloc[j + 1][i - 1] + dT_slice_int 

                # Conduction out
                dT_cond2 = T_cond_int - temps2.iloc[j + 2][i - 1]
                Q_cond2 = (2 * math.pi * k * L * dT_cond2) / (math.log(sliceD[j + 2] / sliceD[j + 1]))
                dT_slice2 = (Q_cond - Q_cond2) * (t[i] - t[i - 1]) / (m[j + 1] * Cp)
                T_slice2 = temps2.iloc[j + 1][i - 1] + dT_slice2
                T.append(T_slice2)
                Q_cond_vec.append(Q_cond2)
                # print(f"m[j+1] = {m[j+1]}") - Correct
                
            else:
                dT_slice_int = Q_cond_vec[j - 1] * (t[i] - t[i - 1]) / (m[j + 1] * Cp) # Mass value will change depending on slice
                T_cond_int = temps2.iloc[j + 1][i - 1] + dT_slice_int 

                # Conduction out
                dT_cond2 = T_cond_int - temps2.iloc[j + 2][i - 1]
                Q_cond2 = (2 * math.pi * k * L * dT_cond2) / (math.log(sliceD[j + 2] / sliceD[j + 1]))
                Q_cond_vec.append(Q_cond2)
                dT_slice2 = (Q_cond_vec[j - 1] - Q_cond2) * (t[i] - t[i - 1]) / (m[j + 1] * Cp)
                T_slice2 = temps2.iloc[j + 1][i - 1] + dT_slice2
                T.append(T_slice2)
            

        # Temp change in slice 3 from conduction - not accounting for natural convection leaving
        dT_slice3_int = Q_cond_vec[r - 3] * (t[i] - t[i - 1]) / (m[r - 1] * Cp)
        T_cond_int2 = temps2.iloc[r - 1][i-1] + dT_slice3_int
        
        # Natural convection from slice 3 to freestream air
        Q_conv_nat = h_nat * A2 * (T_cond_int2 - 298)
        dT_slice3 = (Q_cond_vec[r - 3] - Q_conv_nat) * (t[i] - t[i - 1]) / (m[r - 1] * Cp)
        T_slice3 = temps2.iloc[r - 1][i-1] + dT_slice3
        T.append(T_slice3)
        
        temps2[f"t = {t[i]}"] = T
        
plt.figure()
for i in range(r):
    plt.plot(t, temps2.iloc[i], label = f"Slice Temperature {i + 1}")
plt.xlabel("Time (s)")
plt.ylabel("Temperature (K)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.legend()
plt.grid(visible=True, which='both', axis='both', color='lightsteelblue', linestyle='--', linewidth=0.5)
        
plt.figure()
plt.plot(t,temps2.iloc[0])
plt.xlabel("Time (s)")
plt.ylabel("Inner Wall Temperature (K)")
plt.legend()
plt.grid(visible=True, which='both', axis='both', color='lightsteelblue', linestyle='--', linewidth=0.5)

plt.figure()
plt.plot(t,temps2.iloc[r-1])
plt.xlabel("Time (s)")
plt.ylabel("Outer Wall Temperature (K)")
plt.legend()
plt.grid(visible=True, which='both', axis='both', color='lightsteelblue', linestyle='--', linewidth=0.5)

